# ImpDAR Quad-Polarized ApRES (QpDAR)
## Polarimetric Processing

Here, we define a separate data object, QuadPolData, for polarimetric ApRES processing with four acquisitions. This should be done after the initial processing for each aquisition, so have a look at the opening tutorial notebook first. If individual acquisitions are not already processed, the load function tries to do that automatically.

The main processing functionality within the QuadPol class includes: 
- `rotational_transform`
- `cpe_selection`
- `coherence2d`
- `cpe_phase_gradient`

We overview each of these below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from impdar.lib.ApresData import load_apres,load_quadpol

%matplotlib inline

## Load QuadPol data class

Data can be loaded as a QuadPolData object either from a file that was previously written by ImpDAR (.mat, .h5) or from four ApRES data files in the order 'HH', 'HV', 'VH', 'VV'.
- `load_quadpol(['quadpol.mat'])`
- `load_quadpol(['quadpol_HH.DAT', 'quadpol_HV.DAT', 'quadpol_VH.DAT', 'quadpol_VV.DAT'])`

If you leave off the extension the function will try to figure it out for you. For instance:
- `load_quadpol(['quadpol'])` will load `['quadpol_HH.DAT', 'quadpol_HV.DAT', 'quadpol_VH.DAT', 'quadpol_VV.DAT']`

The load script then checks that all 4 aquisitions are compatible and tries to do the initial processing before bringing them together.

Here, we use data from WAIS Divide (Young et al., 2021).

In [ ]:
# Load the four acquisitions, one for each acquisitions
qp_dat = load_quadpol.load_quadpol('./data/quadpol')

# ----------------------------------------------------------

# Plot all four acquisitions together
plt.figure()

plt.subplot(221)
plt.plot(10.*np.log10(qp_dat.shh**2.),qp_dat.range,'.',c='k',ms=1,alpha=0.2);
plt.ylim(3000,0)
plt.title('shh',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Power (dB)')

plt.subplot(222)
plt.plot(10.*np.log10(qp_dat.shv**2.),qp_dat.range,'.',c='k',ms=1,alpha=0.2);
plt.ylim(3000,0)
plt.title('shv',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Power (dB)')

plt.subplot(223)
plt.plot(10.*np.log10(qp_dat.svh**2.),qp_dat.range,'.',c='k',ms=1,alpha=0.2);
plt.ylim(3000,0)
plt.title('svh',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Power (dB)')

plt.subplot(224)
plt.plot(10.*np.log10(qp_dat.svv**2.),qp_dat.range,'.',c='k',ms=1,alpha=0.2);
plt.ylim(3000,0)
plt.title('svv',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Power (dB)')

plt.tight_layout()

With the stacked data imported into a single data class we then use a rotational transform to simulate data in an azimuthal rotation.
`qp_dat.rotational_transform()`

$$ S = \begin{bmatrix}
s_{hh} & s_{hv}\\
s_{vh} & s_{vv}
\end{bmatrix}$$

$$ S'(\theta) = R(\theta) S R^T(\theta) $$

This takes the number of azimuths, `n_thetas` as input. The function also checks that the cross-polarized terms are consistent with one another. If they were oriented backwards (as has been reported an issue by several field groups) they will have opposite sign and the rotational transform will produce falsely symmetric data.

In [ ]:
# Rotate the acquisitions around all azimuths
qp_dat.rotational_transform(cross_pol_flip='HV')

Θs,Ds = np.meshgrid(qp_dat.thetas,qp_dat.range)

plt.figure()

plt.subplot(221)
plt.contourf(Θs,Ds,10.*np.log10(qp_dat.HH**2.),cmap='Greys_r',levels=25,vmin=-25,vmax=25);
plt.ylim(3000,0)
plt.title('HH',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

plt.subplot(222)
plt.contourf(Θs,Ds,10.*np.log10(qp_dat.HV**2.),cmap='Greys_r',levels=25,vmin=-25,vmax=25);
plt.ylim(3000,0)
plt.title('HV',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

plt.subplot(223)
plt.contourf(Θs,Ds,10.*np.log10(qp_dat.VH**2.),cmap='Greys_r',levels=25,vmin=-25,vmax=25);
plt.ylim(3000,0)
plt.title('VH',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

plt.subplot(224)
plt.contourf(Θs,Ds,10.*np.log10(qp_dat.VV**2.),cmap='Greys_r',levels=25,vmin=-25,vmax=25);
plt.ylim(3000,0)
plt.title('VV',fontweight='bold')
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

plt.tight_layout()

# Find the Girdle Axis

The maximum girdle axis should theoretically be at the location with minimum returned power for cross-polarized acquisitions (Ershadi et al., 2022). Do some filtering of the cross-polarized image (either one) to get the extinction axis.

In [ ]:
from impdar.lib.ApresData._QuadPolProcessing import find_cpe
qp_dat.find_cpe(Wn=20)

plt.figure()
plt.contourf(Θs,Ds,10.*np.log10(qp_dat.HV**2.),cmap='hot',levels=np.linspace(-25,25,21),extend='both')
plt.colorbar()
plt.plot(qp_dat.cpe,qp_dat.range,'m')
plt.ylim(3000,0)
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

# HH-VV Coherence

Compare the two co-polarized images across all azimuths.

In [ ]:
# Do the HH-VV coherence calculation
qp_dat.coherence2d()

Extract the coherence along the cpe axis and get the phase gradient along that axis.

In [ ]:
# Mesh the thetas and ranges in order to reference the filled contour plots
Θs,Ds = np.meshgrid(qp_dat.thetas,qp_dat.range)

plt.figure(figsize=(10,4))
plt.subplot(131)
plt.contourf(Θs,Ds,np.abs(qp_dat.chhvv),cmap='Greys_r',levels=100)
plt.colorbar(label='$|c_{hhvv}|$',orientation='horizontal',ticks=np.arange(0,1.1,0.25))
plt.ylim(2000,0)
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

plt.subplot(132)
plt.contourf(Θs,Ds,np.angle(qp_dat.chhvv),cmap='twilight_shifted',levels=100)
plt.colorbar(label='$\phi_{hhvv}$',orientation='horizontal',ticks=np.arange(-np.pi,np.pi+0.1,np.pi/2.))
plt.ylim(2000,0)
plt.plot(qp_dat.cpe,qp_dat.range,'m')
plt.xlabel('Rotation (rad)')

plt.subplot(133)
plt.plot(np.angle(qp_dat.chhvv_cpe),qp_dat.range,'k.')
plt.gca().set_xticks(np.arange(-np.pi,np.pi+0.1,np.pi/2.))
plt.ylim(2000,0)

plt.tight_layout()

# Phase Gradient

Calculate the gradient of $\phi_{hhvv}$ in the image above.
We use both the imaginary and real components so that we do not need to 'unwrap' manually (Jordan et al., 2019)

$$ $$

In [ ]:
# Do the HH-VV coherence calculation
qp_dat.phase_gradient2d(Wn=50,filt='lowpass')

In [ ]:
plt.figure()

plt.subplot(121)
plt.contourf(Θs,Ds,qp_dat.dphi_dz,cmap='seismic',levels=np.linspace(-.02,.02,50),extend='both')
plt.colorbar(label='$\partial \phi /\partial z$',orientation='horizontal',ticks=np.arange(-0.02,0.021,0.01))
plt.plot(qp_dat.cpe,qp_dat.range,'m')
plt.ylim(2000,0)
plt.ylabel('Range (m)')
plt.xlabel('Rotation (rad)')

plt.subplot(122)
qp_dat.dphi_dz_cpe = qp_dat.dphi_dz[np.arange(qp_dat.snum), qp_dat.cpe_idxs]
plt.plot(qp_dat.dphi_dz_cpe,qp_dat.range,'k.',alpha=0.01)
plt.xlim(-0.02,0.02)
plt.ylim(2000,0)

plt.tight_layout()